# Setup

In [ ]:
!pip install -qU langchain
!pip install -qU langchain-google-genai
!pip install -qU langchain-huggingface
!pip install -qU langchain-qdrant
!pip install -qU langchain-community
!pip install -qU langgraph
!pip install fastembed

zsh:1: /Users/idhibhatpankam/Code/chula/capstone/grader-rag/.venv/bin/pip: bad interpreter: /Users/idhibhatpankam/Code/capstone/grader-rag/.venv/bin/python3: no such file or directory
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-community 0.3.12 requires langsmith<0.3,>=0.1.125, but you have langsmith 0.3.18 which is incompatible.
zsh:1: /Users/idhibhatpankam/Code/chula/capstone/grader-rag/.venv/bin/pip: bad interpreter: /Users/idhibhatpankam/Code/capstone/grader-rag/.venv/bin/python3: no such file or directory
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.4 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.
zsh:1: /Users/idhibhatp

In [2]:
import os
from dotenv import load_dotenv
import pandas as pd
from pandas import DataFrame
import numpy as np
from tqdm import tqdm

load_dotenv()

# for colab
# from google.colab import userdata

# os.environ["GOOGLE_API_KEY"] = userdata.get('gemini_api_key')
# os.environ["HUGGINGFACE_API_KEY"] = userdata.get('huggingface_api_key')
# os.environ["LANGSMITH_API_KEY"] = userdata.get('langsmith_api_key')
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGSMITH_TRACING"] = "true"
# os.environ["LANGSMITH_PROJECT"] = "nlp_project"

True

In [3]:
from huggingface_hub import login

login(token=os.getenv("HUGGINGFACE_API_KEY"))
print("Logged in successfully!")

/Users/idhibhatpankam/Code/chula/capstone/grader-rag/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Logged in successfully!


In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-001")

In [5]:
llm.invoke("hi")

AIMessage(content='Hi there! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-eebe3ff4-9014-472e-82f0-3fdcdb7dadcd-0', usage_metadata={'input_tokens': 1, 'output_tokens': 11, 'total_tokens': 12, 'input_token_details': {'cache_read': 0}})

In [6]:
from langchain_huggingface import HuggingFaceEmbeddings

model_kwargs = {'trust_remote_code': True}
embeddings = HuggingFaceEmbeddings(model_name="Alibaba-NLP/gte-modernbert-base", model_kwargs=model_kwargs)

In [7]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder

model = HuggingFaceCrossEncoder(model_name="Alibaba-NLP/gte-reranker-modernbert-base")

## Loading data
- only include submissions with full score
- duplicate code (exact match) is not allowed for unknown/generated code
- duplicate code (only different variable names) is not allowed for human-written code

In [8]:
submissions = pd.read_csv("./data/submissions.csv")
submissions.groupby(["problem_id", "type"]).size().unstack(fill_value=0)

type,human,unknown
problem_id,,
05_List_11,2029,1304
08_Dict_22,2018,414


In [9]:
generated_submissions = pd.read_csv("./data/generated_submissions.csv")
generated_submissions.groupby("problem_id").size().reset_index(name="count")

,problem_id,count
0,05_List_11,62
1,08_Dict_22,378


In [10]:
submissions = pd.concat([submissions, generated_submissions], ignore_index=True)
submissions.groupby(["problem_id", "type"]).size().unstack(fill_value=0)

type,generated,human,unknown
problem_id,,,
05_List_11,62,2029,1304
08_Dict_22,378,2018,414


In [11]:
submissions

,id,author,problem_id,code,submission_id,score,time,type
0,1255109.0,6330139021,05_List_11,x = input()\na = [str(i) for i in range(10)]\n...,1255109.0,100.0,over 4 years ago,human
1,1257085.0,6330004021,05_List_11,"n=input()\nans=''\ndg=['0','1','2','3','4','5'...",1257085.0,100.0,over 4 years ago,human
2,1255424.0,6330015921,05_List_11,"a=input()\nb=[]\nc=""""\nfor ch in a:\n if""0""...",1255424.0,100.0,over 4 years ago,human
3,1047529.0,6230080521,05_List_11,"a=input().strip()\nn=['0','1','2','3','4','5',...",1047529.0,100.0,over 5 years ago,human
4,1253956.0,6231007021,05_List_11,"dg = ['0','1','2','3','4','5','6','7','8','9']...",1253956.0,100.0,over 4 years ago,human
...,...,...,...,...,...,...,...,...
6200,NaN,gemini-2.0-flash-001,08_Dict_22,def solve():\n n = int(input())\n ice_cr...,NaN,NaN,2025-03-21 19:43:16.459842,generated
6201,NaN,gemini-2.0-flash-001,08_Dict_22,def solve():\n n = int(input())\n prices...,NaN,NaN,2025-03-21 19:43:18.512018,generated
6202,NaN,gemini-2.0-flash-001,08_Dict_22,def solve():\n n = int(input())\n prices...,NaN,NaN,2025-03-21 19:43:20.765874,generated
6203,NaN,gemini-2.0-flash-001,08_Dict_22,def solve():\n n = int(input())\n ice_cr...,NaN,NaN,2025-03-21 19:43:49.863853,generated


In [12]:
rag_submissions = submissions[submissions["type"].isin(["human", "generated"])]
rag_submissions.groupby(["problem_id", "type"]).size().unstack(fill_value=0)

type,generated,human
problem_id,,
05_List_11,62,2029
08_Dict_22,378,2018


In [13]:
from split_df import split_dataframe

train_df, val_df, test_df = split_dataframe(rag_submissions, stratify_cols=["problem_id", "type"], random_state=42)

train_df.shape, val_df.shape, test_df.shape

/Users/idhibhatpankam/Code/chula/capstone/grader-rag/src/split_df.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["_stratify_group"] = df[stratify_cols].astype(str).agg("_".join, axis=1)


((3589, 8), (449, 8), (449, 8))

In [14]:
train_counts = train_df.groupby(["problem_id", "type"]).size().unstack(fill_value=0)
val_counts = val_df.groupby(["problem_id", "type"]).size().unstack(fill_value=0)
test_counts = test_df.groupby(["problem_id", "type"]).size().unstack(fill_value=0)

combined = pd.concat(
    [train_counts, val_counts, test_counts],
    axis=1,
    keys=["Train", "Validation", "Test"]
)

print(combined)

               Train       Validation            Test      
type       generated human  generated human generated human
problem_id                                                 
05_List_11        50  1623          6   203         6   203
08_Dict_22       302  1614         38   202        38   202


## Retrievers

In [36]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
from langchain_core.documents import Document

# code for indexing is in rag notebook
client = QdrantClient(host="localhost", port=6333)

vector_store = QdrantVectorStore(
    client=client,
    collection_name="submission_collection",
    embedding=embeddings,
)

/var/folders/27/0_8pjdb52g91gmnzbtgblvgc0000gn/T/ipykernel_97571/1119695003.py:7: UserWarning: Qdrant client version 1.13.3 is incompatible with server version 1.9.2. Major versions should match and minor version difference must not exceed 1. Set check_version=False to skip version check.
  client = QdrantClient(host="localhost", port=6333)


In [34]:
retriever_normal = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 20},  # top_k 20
)

# Maximal Marginal Relevance (MMR) is a hybrid search
retriever_mmr = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 20, "fetch_k": 20, "lambda_mult": 0.5},
)

reranker = CrossEncoderReranker(model=model)
def reranked_retriever(query):
    # retrieve top 20 docs
    retrieved_docs = retriever_mmr.invoke(query)

    # rerank 20 docs, keep only top 3
    reranked_docs = reranker.compress_documents(retrieved_docs, query=query)[:3]

    return reranked_docs


query = generated_submissions.iloc[0]["code"]

print("Normal Retriever Results:")
res_normal = retriever_normal.invoke(query)
for i in range(3):
    print(res_normal[i].metadata)

print("\nMMR Retriever Results:")
res_mmr = retriever_mmr.invoke(query)
for i in range(3):
    print(res_mmr[i].metadata)

print("\nReranked Retriever Results:")
res_reranked = reranked_retriever(query)
for i in range(min(3, len(res_reranked))):
    print(res_reranked[i].metadata)

Normal Retriever Results:


UnexpectedResponse: Unexpected Response: 404 (Not Found)
Raw response content:
b''

In [33]:
generated_submissions.iloc[0]

id                                                             NaN
author                                        gemini-2.0-flash-001
problem_id                                              05_List_11
code             s = input()\nmissing = []\nfor i in range(10):...
submission_id                                                  NaN
score                                                          NaN
time                                    2025-04-28 22:43:32.473922
type                                                     generated
Name: 0, dtype: object

## Retrieval Evaluation

Calculate the MRR of the two retrievers based on the top 3 returned by each retriever.

In [27]:
def cosine_similarity(vec1, vec2):
    vec1 = np.array(vec1)
    vec2 = np.array(vec2)
    dot_product = np.dot(vec1, vec2)

    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    similarity = dot_product / (norm_vec1 * norm_vec2)

    return similarity

In [ ]:
columns = [
        "submission_id", "code", "author", "type",
        "retrieved_code", "retrieved_author", "retrieved_type", "similarity"
    ]

retrieved_docs_normal = pd.DataFrame(columns=columns)
retrieved_docs_mrr = pd.DataFrame(columns=columns)
retrieved_docs_reranked = pd.DataFrame(columns=columns)

In [ ]:
limit = -1
count = 0
hit_normal = miss_normal = 0

for _, row in tqdm(val_df.iterrows()):
    submission_id = row["submission_id"]
    code = row["code"]
    author = row["author"]
    _type = row["type"]

    res_normal = retriever_normal.invoke(query)
    similarities = []
    for i in range(3):
        retrieved_author = res_normal[i].metadata["author"]
        retrieved_code = res_normal[i].page_content
        retrieved_type = res_normal[i].metadata["type"]

        val_embed = embeddings.embed_query(code)
        doc_embed = embeddings.embed_query(retrieved_code)
        similarity = cosine_similarity(val_embed, doc_embed)
        similarities.append(similarity)

        retrieved_docs_normal.loc[len(retrieved_docs_mrr)] = [
            submission_id, code, author, _type, 
            retrieved_code, retrieved_author, retrieved_type, similarity]    

    found = False
    for i in range(3):
        if similarities[i] > 0.9: # sim 0.9 not found only 2 times
            hit_mrr += (1/(i+1)) * similarity
            found = True
            break
    if not found:
        print(f"Query {count}: Failed to find relevant doc in top 3 results")
        miss_mrr += 1

    count += 1
    if limit != -1 and count == limit:
        break

print(f"Hit NORMAL: {hit_normal}")
print(f"Miss NORMAL: {miss_normal}")

In [ ]:
retrieved_docs_normal.to_csv("retrieved_docs_normal.csv", index=False)

In [ ]:
limit = -1
count = 0
hit_mrr = miss_mrr = 0

for _, row in tqdm(val_df.iterrows()):
    submission_id = row["submission_id"]
    code = row["code"]
    author = row["author"]
    _type = row["type"]

    res_mmr = retriever_mmr.invoke(code)
    similarities = []
    for i in range(3):
        retrieved_author = res_mmr[i].metadata["author"]
        retrieved_code = res_mmr[i].page_content
        retrieved_type = res_mmr[i].metadata["type"]

        val_embed = embeddings.embed_query(code)
        doc_embed = embeddings.embed_query(retrieved_code)
        similarity = cosine_similarity(val_embed, doc_embed)
        similarities.append(similarity)

        retrieved_docs_mrr.loc[len(retrieved_docs_mrr)] = [
            submission_id, code, author, _type, 
            retrieved_code, retrieved_author, retrieved_type, similarity]    

    found = False
    for i in range(3):
        if similarities[i] > 0.9: # sim 0.9 not found only 2 times
            hit_mrr += (1/(i+1)) * similarity
            found = True
            break
    if not found:
        print(f"Query {count}: Failed to find relevant doc in top 3 results")
        miss_mrr += 1

    count += 1
    if limit != -1 and count == limit:
        break

print(f"Hit MRR: {hit_mrr}")
print(f"Miss MRR: {miss_mrr}")

0it [00:00, ?it/s]

79it [07:26,  5.68s/it]

Query 78: Failed to find relevant doc in top 3 results


106it [10:05,  7.11s/it]

In [ ]:
retrieved_docs_mrr.to_csv("retrieved_docs_mrr.csv", index=False)

In [ ]:
limit = -1
count = 0
hit_reranked = miss_reranked = 0

for _, row in tqdm(val_df.iterrows()):
    submission_id = row["submission_id"]
    code = row["code"]
    author = row["author"]
    _type = row["type"]

    res_reranked = reranked_retriever(code)
    similarities = []
    for i in range(3):
        retrieved_author = res_reranked[i].metadata["author"]
        retrieved_code = res_reranked[i].page_content
        retrieved_type = res_reranked[i].metadata["type"]

        val_embed = embeddings.embed_query(code)
        doc_embed = embeddings.embed_query(retrieved_code)
        similarity = cosine_similarity(val_embed, doc_embed)
        similarities.append(similarity)

        retrieved_docs_reranked.loc[len(retrieved_docs_reranked)] = [
            submission_id, code, author, _type, 
            retrieved_code, retrieved_author, retrieved_type, similarity]    

    found = False
    for i in range(3):
        if similarities[i] > 0.9: # sim 0.9 not found only 2 times
            hit_reranked += (1/(i+1)) * similarity
            found = True
            break
    if not found:
        print(f"Query {count}: Failed to find relevant doc in top 3 results")
        miss_reranked += 1

    count += 1
    if limit != -1 and count == limit:
        break

print(f"Hit RERANKED: {hit_reranked}")
print(f"Miss RERANKED: {miss_reranked}")

In [ ]:
retrieved_docs_mrr.to_csv("retrieved_docs_reranked.csv", index=False)

In [ ]:
# limit = -1
# count = 0
# rerank = rerank_failures = 0

# for code in tqdm(sus_submissions["code"]):
#     res_reranked = reranked_retriever(code)
#     found = False
#     for i in range(3):
#         id = res_reranked[i].metadata["_id"]
#         doc_content = res_reranked[i].page_content

#         sus_embed = embeddings.embed_query(code)
#         doc_embed = embeddings.embed_query(doc_content)
#         similarity = cosine_similarity(sus_embed, doc_embed)
#         if similarity > 0.9:
#             rerank += (1/(i+1)) * similarity
#             found = True
#             break
#     if not found:
#         print(f"Query {count}: Failed to find relevant doc in top 3 results")
#         rerank_failures += 1

#     count += 1
#     if limit != -1 and count == limit:
#         break

100%|██████████| 2174/2174 [20:54<00:00,  1.73it/s]


In [ ]:
sample_size = val_df.shape[0]

# this MRR = mean reciprocal rank
print(f"MRR of normal retriever: {hit_normal / sample_size:.4f}")
print(f"MRR of MRR retriever: {hit_mrr / sample_size:.4f}")
print(f"MRR of reranked retriever: {hit_reranked / sample_size:.4f}")

print(f"Percentage of queries with no relevant doc in top 3 (normal): {100 * miss_normal / sample_size:.2f}%")
print(f"Percentage of queries with no relevant doc in top 3 (MMR): {100 * miss_mrr / sample_size:.2f}%")
print(f"Percentage of queries with no relevant doc in top 3 (rerank): {100 * miss_reranked / sample_size:.2f}%")

MRR of the retriever without a reranker: 0.9650
MRR of the retriever with a reranker: 0.9548
Percentage of queries with no relevant doc in top 3 (no rerank): 0.00%
Percentage of queries with no relevant doc in top 3 (rerank): 0.00%
